# Introduction

### Get GPS Result Files 
1.  The current version of human proteome conanical sequences were downloaded from [Uniprit.org](https://www.uniprot.org) 
    - use CreateHumanProteomeDF.ipynb
    - Or, go [HERE](https://www.uniprot.org/uniprot/?query=*&fil=organism%3A%22Homo+sapiens+%28Human%29+%5B9606%5D%22+AND+reviewed%3Ayes), click on 'Download'-> FASTA(canonical)
2.  split the downloaded fasta file into smaller files with 1000 sequence in each. The last file has 367 sequences. (CreateHumanProteomeDF.ipynb)
3. Download the GPS softwarefor your operating system from [GPS downloading page](http://gps.biocuckoo.cn/download.php). Follow the user prompts through the installation.
4.  Follow the instruction of 'Batch Prediction' on page 8 of the [User Guide](http://gps.biocuckoo.cn/download/GPS_5.0_Manual_2019.pdf) 
5.  submit the sequence files in GPS BatchPredictor with the following settings:
    - **PKs**: select all kinases
    - **Sequence File List:** add the sequence fasta files
    - **result Export Folder:** choose the dir to save the output files ('../Data/Raw/GPS/')
    - **Threshold:** all 
    

### About the data
- Version: Nov., 2019
- GPS will take about 10 days to complete the prediction all the human proteomes
- raw predictions are available upon request
        
### Preprocessing GPS Raw
In order to convert this to standard format, here we:
1. Convert the GPS predictors in the manually created gps_predictor.csv file to kinases with kinase_acc (UniprotID).
    - input files:  **'gps_predictor.csv'**
    - output file: **'gps_predictor.csv'** output data frame with the following columns:
       - **predictor** - GPS predictors for each kinase
       - **kinase**	- kinase names used in GPS
       - **kinase_acc** - UniprotIDs of the kinases
    
1. Convert the result files to a data frame containing following columns, and combine the 21 result files into one GPS file: 
    - **Mapping Substrate Accessions (UniprotID)** Extract the substrate UniprotID from the idendifiers of the submitted sequences. Create the the 'substrate_acc' column.
    - **Mapping Kinase Accessions (UniprotID)** Merge the converted data frame with the manually generated 'gps_valid_kinases.csv'
    - output data frame with the following columns:
        - **substrate_acc** - UniprotIDs extracted from the idendifiers of the submitted sequences
        - **site** -  'residue' + 'position'
        - **residue** - AA at the phosphorylated position
        - **position** - position in the protein sequence
        - **predictor** - GPS predictor
        - **pep** - +/- 7AA peptides
        - **score**
        - **threshold**
        - **kinase** - kinase name used in GPS
        - **kinase_acc** - UniprotIDs of the kinases
    
        - input files:  21 GPS result files 
        - output file: **'GPS_temp_mappedAcc.csv'** 

    - **Mapping sites** Map the phosphorylation site relative to the current human proteome sequence. This will filter out the outdated records.
        - input file:  'GPS_temp_mappedAcc.csv
        - final output file: **'GPS_temp_mappedSite.csv'** added/updated columns:
            - **substrate_id** - unique IDs for the substrate phosphorylation site (substrate_acc + position)
            - **site** - mapped phosphorylation site

    

### Creating Resource Files
1.  **'globalKinaseMap.csv':** After step 1 in preprocessing, add unique kinases from GPS to the globel kinase resource. 
    

### Standard Formatted GPS
**'GPS.csv':** Standardize the preprocessed file with following columns:
- **substrate_id** - unique IDs for the substrate phosphorylation site (substrate_acc + position)
- **substrate** - gene name for the substrates
- **substrate_acc** - mapped UniprotIDs for the substrates
- **site** - phosphorylation  site
- **pep** - +/- 7 AA peptide sequence around the site
- **kinase** - Kinase name


# Initializing

In [1]:
# IMPORTS
#%pip install 
import pandas as pd
import glob
import os
import time
import re

import humanProteomesReference, gps_convert, getUniprotID, createSubKinMatrix

In [3]:
# DEFINE FILE NAMES/DIRs 
##################
# Version (Date) #
##################
version = '2019-12-11'

##################
# File Location  #
##################
# local (../../)
base = '../../'

###############################################
# For Prepare Fasta Files to Submit in GPS #
###############################################
# Human Proteome fasta file
HP_fasta = base + 'Data/Raw/HumanProteome/humanProteome_' + version + '.fasta'
# Dir for splited Human Proteome fasta files
HP_dir = base + 'Data/Raw/HumanProteome/'


###############################################
# For Preprocessing GPS Prediction Results #
###############################################

# human proteome referece file 
HP_csv = base + 'Data/Map/humanProteome_' + version + '.csv'

# manually prepared GPS perdictor table
GPS_predictor = base + 'Data/Raw/GPS/gps_predictor.csv'
GPS_kinase = base + 'Data/Raw/GPS/gps_valid_kinases.csv'

# GPS results dir
GPS_dir = base + 'Data/Raw/GPS/'
GPS_update_dir = base + 'Data/Raw/GPS/updated/'

# GPS temp dir
GPS_temp_dir_acc = base + 'Data/Temp/GPS/mappedAcc/'
GPS_temp_dir_site = base + 'Data/Temp/GPS/mappedSite/'
GPS_temp_dir_acc_update = base + 'Data/Temp/GPS/mappedAcc/updated/'
GPS_temp_dir_site_update = base + 'Data/Temp/GPS/mappedSite/updated/'

# Resource Files
HK_org = base + 'Data/Raw/HumanKinase/globalKinaseMap.txt'                  # orginal manually created kinase file
KinaseMap = base + 'Data/Map/globalKinaseMap.csv'                           # add all unique kinase in HPRD to the global file

# Standard formatted output file
GPS_formatted = base + 'Data/Formatted/GPS/GPS_formatted_' + version + '.csv'       # preprocessed file with cloumns: substrate_id/substrate/substrate_acc/kinase/site/pep/score


# Preprocessing GPS Raw Predictions
### Convert GPS predictor
Convert the GPS predictors in the manually created gps5_predictor.csv file to kinases with kinase_acc (UniprotID).

In [3]:
# convert the predictor to kinases with kinase_acc
gps_predictor = pd.read_csv(GPS_predictor)
# the individual kinase name is after the last '/' in the predictor name (e.g. the kinase predicted by 'AGC/Akt/AKT1' is 'AKT1')
gps_predictor['kinase'] = gps_predictor.apply(lambda row :  re.search(r'.+\/(.+)$', str(row['predictor'])).group(1), axis = 1) 
# get the uniprotID of the kinases
gps_predictor['kinase_acc'] = gps_predictor.apply(lambda row :  getUniprotID.getUniprotID(str(row['kinase']), 'gene name'), axis = 1) 
gps_predictor

CBK1 no hit in human
PKH1 no hit in human
PKH2 no hit in human
SRK2E no hit in human
MLKA no hit in human
DUN1 no hit in human
CDC28 no hit in human
PHO85 no hit in human
GSK3 no hit in human
FUS3 no hit in human
HOG1 no hit in human
CLA4 no hit in human
STE20 no hit in human
TOR2 no hit in human
RIO no hit in human
AurB no hit in human
IPL1 no hit in human
CDC5 no hit in human
EPHB5 no hit in human
AGC no hit in human
Aur no hit in human


,predictor,kinase,kinase_acc
0,AGC/Akt/AKT1,AKT1,P31749
1,AGC/Akt/AKT2,AKT2,P31751
2,AGC/Akt/AKT3,AKT3,Q9Y243
3,AGC/DMPK/CRIK/CIT,CIT,O14578
4,AGC/DMPK/GEK/DMPK,DMPK,Q09013
...,...,...,...
359,Dual/TKL/STKR/STKR2/TGFBR2,TGFBR2,P37173
360,Dual/Other/Aur,Aur,(no hit in human)
361,Dual/Other/PEK/PKR/EIF2AK2,EIF2AK2,P19525
362,Dual/Other/WEE/WEE1/WEE1B,WEE1B,P0C1S8


Create a dictionary of the human kinases that fail to retrieve Programmatically. Using the dictionary to make the chages in the gps_kinases dataframe.

In [4]:
new_kinase = {'AurB' : 'Q96GD4'}
# Manually add the uniprotID for the above kinase
for key in new_kinase:
    gps_predictor.loc[gps_predictor.kinase == key, ["kinase_acc"]] = new_kinase[key]
# remove non-human kinases
gps_predictor = gps_predictor[gps_predictor['kinase_acc'] != '(no hit in human)']

gps_predictor

,predictor,kinase,kinase_acc
0,AGC/Akt/AKT1,AKT1,P31749
1,AGC/Akt/AKT2,AKT2,P31751
2,AGC/Akt/AKT3,AKT3,Q9Y243
3,AGC/DMPK/CRIK/CIT,CIT,O14578
4,AGC/DMPK/GEK/DMPK,DMPK,Q09013
...,...,...,...
358,Dual/TKL/LRRK/LRRK2,LRRK2,Q5S007
359,Dual/TKL/STKR/STKR2/TGFBR2,TGFBR2,P37173
361,Dual/Other/PEK/PKR/EIF2AK2,EIF2AK2,P19525
362,Dual/Other/WEE/WEE1/WEE1B,WEE1B,P0C1S8


get a list Kinases that are predicted by multiple predictors. Get the indexs of the ones predict by the Dual predictors, remove them from the gps_kinases, save the file.

In [5]:
duplicateKinase = gps_predictor[gps_predictor.duplicated(['kinase'], keep = False)].sort_values(by=['kinase'])
duplicateKinase = duplicateKinase[(duplicateKinase['predictor'].str.contains('Dual'))]
idx_li = duplicateKinase.index.values.tolist()
idx_li

[278,
 346,
 349,
 350,
 361,
 358,
 353,
 354,
 356,
 355,
 357,
 351,
 352,
 347,
 275,
 276,
 277,
 359]

In [6]:
# remove the Dual predictor of the kinase predicted by multipule GPS predictor
for idx in idx_li:
    gps_predictor = gps_predictor.drop(idx)
# save the file
gps_predictor.to_csv(GPS_kinase, index=False)
gps_predictor

,predictor,kinase,kinase_acc
0,AGC/Akt/AKT1,AKT1,P31749
1,AGC/Akt/AKT2,AKT2,P31751
2,AGC/Akt/AKT3,AKT3,Q9Y243
3,AGC/DMPK/CRIK/CIT,CIT,O14578
4,AGC/DMPK/GEK/DMPK,DMPK,Q09013
...,...,...,...
343,TK/VEGFR/KDR,KDR,P35968
345,Dual/Atypical/BAZ/BAZ1B,BAZ1B,Q9UIG0
348,Dual/CMGC/CK2/CSNK2A1,CSNK2A1,P68400
362,Dual/Other/WEE/WEE1/WEE1B,WEE1B,P0C1S8


### Convert GPS Data Frame

- convert the fasta like GPS result files to a pandas data frame
- **Mapping Substrate Accessions (UniprotID)** Extract the substrate UniprotID from the idendifiers of the submitted sequences. Create the the 'substrate_acc' column.
- **Mapping Kinase Accessions (UniprotID)** Merge the converted data frame with the manually generated 'gps_valid_kinases.csv' with  following coulmns:
    - **predictor** - list of GPS predictor
    - **kinase** - the name of the kinase used in GPS
    - **is_kinase** - 0 (family or group) or 1 (kinase), **this file ONLY contains perdictions of kinase, kinase group/family are excluded.**
    - **kinase_acc** - programmatically retreived UniprotID from the given kinase names with manual confirmation. (the ones that are not human kinase or no hit in Uniprot.org are '(no hit in human)' )
- Combine the 'residue' and 'position' to create the 'site' column
- the output df contains the following columns:
    - **substrate_acc** - UniprotIDs extracted from the idendifiers of the submitted sequences
    - **site** -  'residue' + 'position'
    - **residue** - AA at the phosphorylated position
    - **position** - position in the protein sequence
    - **predictor** - GPS predictor
    - **pep** - +/- 7AA peptides
    - **score**
    - **threshold**
    - **kinase** - kinase name used in GPS
    - **kinase_acc**


In [7]:
# create a data frame from the GPS raw, add substrate and kinase preotien acc (uniprotID)
convert_type = 'acc'
gps_convert.gps_convert_directory(GPS_dir, GPS_kinase, GPS_temp_dir_acc, convert_type)

Formatting  group_2 ...
Done. Time	1518.487
Formatting  group_17 ...
Done. Time	1558.139
Formatting  group_4 ...
Done. Time	1710.565
Formatting  group_9 ...
Done. Time	1631.413
Formatting  group_11 ...
Done. Time	1962.357
Formatting  group_3 ...
Done. Time	1925.417
Formatting  group_16 ...
Done. Time	2118.729
Formatting  group_5 ...
Done. Time	2445.987
Formatting  group_8 ...
Done. Time	2148.243
Formatting  group_10 ...
Done. Time	2218.017
Formatting  group_18 ...
Done. Time	2198.258
Formatting  group_20 ...
Done. Time	2143.964
Formatting  group_15 ...
Done. Time	2150.675
Formatting  group_6 ...
Done. Time	2372.631
Formatting  group_13 ...
Done. Time	2279.974
Formatting  group_1 ...
Done. Time	2274.745
Formatting  group_19 ...
Done. Time	2372.402
Formatting  group_21 ...
Done. Time	358.565
Formatting  group_14 ...
Done. Time	1977.981
Formatting  group_7 ...
Done. Time	2007.099
Formatting  group_12 ...
Done. Time	2388.232


### Mapping Sites against the current human Proteomes
- get the +/- 7 AA peptide using the position from the referece human Proteomes csv file
- compare the retrieved peptide sequence with the provied peptide sequence in the GPS df
- assign substrate_id (substrate_acc + position) to the ones with matched pep seq, 
- assign substrate_id as 'outdated' to the ones that are not matched

In [8]:
# map the site to the reference human proteome seq (version at the time is Dec. 2019)
convert_type = 'site'
gps_convert.gps_convert_directory(GPS_temp_dir_acc, HP_csv, GPS_temp_dir_site, convert_type)

Formatting  group_21.csv ...
Reading input file...
Get unique substrate sites...
Map unique substrate sites...
Done. Time	96.451
Formatting  group_8.csv ...
Reading input file...
Get unique substrate sites...
Map unique substrate sites...
Q29974 :   corrected:  (idNotFound)10  site:  10 pep:  CLKLPGGSCMTALTV
Q29974 :   corrected:  (idNotFound)13  site:  13 pep:  LPGGSCMTALTVTLM
Q29974 :   corrected:  (idNotFound)16  site:  16 pep:  GSCMTALTVTLMVLS
Q29974 :   corrected:  (idNotFound)18  site:  18 pep:  CMTALTVTLMVLSSP
Q29974 :   corrected:  (idNotFound)23  site:  23 pep:  TVTLMVLSSPLALAG
Q29974 :   corrected:  (idNotFound)24  site:  24 pep:  VTLMVLSSPLALAGD
Q29974 :   corrected:  (idNotFound)32  site:  32 pep:  PLALAGDTRPRFLWQ
Q29974 :   corrected:  (idNotFound)50  site:  50 pep:  ECHFFNGTERVRFLD
Q29974 :   corrected:  (idNotFound)66  site:  66 pep:  YFYNQEESVRFDSDV
Q29974 :   corrected:  (idNotFound)71  site:  71 pep:  EESVRFDSDVGEYRA
Q29974 :   corrected:  (idNotFound)80  site:  80 pe

P49895 :   corrected:  (seqNotFound)123  site:  123  seq len:  249 pep:  PLVLNFGSCTPSFMF
P49895 :   corrected:  (seqNotFound)125  site:  125  seq len:  249 pep:  VLNFGSCTPSFMFKF
P49895 :   corrected:  (seqNotFound)127  site:  127  seq len:  249 pep:  NFGSCTPSFMFKFDQ
Q30167 :   corrected:  (idNotFound)66  site:  66 pep:  RVHNQEEYARYDSDV
Q30167 :   corrected:  (idNotFound)69  site:  69 pep:  NQEEYARYDSDVGEY
Q30167 :   corrected:  (idNotFound)76  site:  76 pep:  YDSDVGEYRAVTELG
Q30167 :   corrected:  (idNotFound)89  site:  89 pep:  LGRPDAEYWNSQKDL
Q30167 :   corrected:  (idNotFound)107  site:  107 pep:  RRAAVDTYCRHNYGV
Q30167 :   corrected:  (idNotFound)112  site:  112 pep:  DTYCRHNYGVGESFT
Q30167 :   corrected:  (idNotFound)131  site:  131 pep:  VQPKVTVYPSKTQPL
Q30167 :   corrected:  (idNotFound)152  site:  152 pep:  VCSVNGFYPGSIEVR
Q30167 :   corrected:  (idNotFound)200  site:  200 pep:  VPQSGEVYTCQVEHP
Q30167 :   corrected:  (idNotFound)249  site:  249 pep:  LGAGLFIYFRNQKGH
Done. Time	

Done. Time	237.118
Formatting  group_18.csv ...
Reading input file...
Get unique substrate sites...
Map unique substrate sites...
P13761 :   corrected:  (idNotFound)10  site:  10 pep:  CLKLPGGSCMAALTV
P13761 :   corrected:  (idNotFound)16  site:  16 pep:  GSCMAALTVTLMVLS
P13761 :   corrected:  (idNotFound)18  site:  18 pep:  CMAALTVTLMVLSSP
P13761 :   corrected:  (idNotFound)23  site:  23 pep:  TVTLMVLSSPLALAG
P13761 :   corrected:  (idNotFound)24  site:  24 pep:  VTLMVLSSPLALAGD
P13761 :   corrected:  (idNotFound)32  site:  32 pep:  PLALAGDTQPRFLWQ
P13761 :   corrected:  (idNotFound)50  site:  50 pep:  KCHFFNGTERVQFLE
P13761 :   corrected:  (idNotFound)71  site:  71 pep:  EEFVRFDSDVGEYRA
P13761 :   corrected:  (idNotFound)80  site:  80 pep:  VGEYRAVTELGRPVA
P13761 :   corrected:  (idNotFound)89  site:  89 pep:  LGRPVAESWNSQKDI
P13761 :   corrected:  (idNotFound)92  site:  92 pep:  PVAESWNSQKDILED
P13761 :   corrected:  (idNotFound)106  site:  106 pep:  DRRGQVDTVCRHNYG
P13761 :   corre

Get unique substrate sites...
Map unique substrate sites...
Q9BQE4 :   corrected:  (seqNotFound)184  site:  184  seq len:  189 pep:  RPGRRGPSSGGG
Q9BQE4 :   corrected:  (seqNotFound)185  site:  185  seq len:  189 pep:  PGRRGPSSGGG
Q86VQ6 :   corrected:  (seqNotFound)637  site:  637  seq len:  643 pep:  KSSGLDITQKGCG
Done. Time	238.172
Formatting  group_14.csv ...
Reading input file...
Get unique substrate sites...
Map unique substrate sites...
P59797 :   corrected:  (seqNotFound)268  site:  268  seq len:  346 pep:  KRVLIRVTYCGLSYS
P59797 :   corrected:  (seqNotFound)273  site:  273  seq len:  346 pep:  RVTYCGLSYSLRYIL
P59797 :   corrected:  (seqNotFound)275  site:  275  seq len:  346 pep:  TYCGLSYSLRYILLK
P62341 :   corrected:  (seqNotFound)48  site:  48  seq len:  195 pep:  LKFQICVSGYRRVFE
P59797 :   corrected:  (seqNotFound)269  site:  269  seq len:  346 pep:  RVLIRVTYCGLSYSL
P59797 :   corrected:  (seqNotFound)274  site:  274  seq len:  346 pep:  VTYCGLSYSLRYILL
P59797 :   corrected

Q9TQE0 :   corrected:  (idNotFound)10  site:  10 pep:  CLKLPGGSCMAALTV
Q9TQE0 :   corrected:  (idNotFound)16  site:  16 pep:  GSCMAALTVTLMVLS
Q9TQE0 :   corrected:  (idNotFound)18  site:  18 pep:  CMAALTVTLMVLSSP
Q9TQE0 :   corrected:  (idNotFound)23  site:  23 pep:  TVTLMVLSSPLALAG
Q9TQE0 :   corrected:  (idNotFound)24  site:  24 pep:  VTLMVLSSPLALAGD
Q9TQE0 :   corrected:  (idNotFound)32  site:  32 pep:  PLALAGDTQPRFLKQ
Q9TQE0 :   corrected:  (idNotFound)50  site:  50 pep:  ECHFFNGTERVRYLH
Q9TQE0 :   corrected:  (idNotFound)71  site:  71 pep:  EENVRFDSDVGEYRA
Q9TQE0 :   corrected:  (idNotFound)80  site:  80 pep:  VGEYRAVTELGRPVA
Q9TQE0 :   corrected:  (idNotFound)89  site:  89 pep:  LGRPVAESWNSQKDF
Q9TQE0 :   corrected:  (idNotFound)92  site:  92 pep:  PVAESWNSQKDFLER
Q9TQE0 :   corrected:  (idNotFound)106  site:  106 pep:  RRRAEVDTVCRHNYG
Q9TQE0 :   corrected:  (idNotFound)117  site:  117 pep:  HNYGVGESFTVQRRV
Q9TQE0 :   corrected:  (idNotFound)119  site:  119 pep:  YGVGESFTVQRRVHP


Done. Time	266.044
Formatting  group_10.csv ...
Reading input file...
Get unique substrate sites...
Map unique substrate sites...
P07203 :   corrected:  (seqNotFound)47  site:  47  seq len:  203 pep:  LLIENVASLGTTVRD
P07203 :   corrected:  (seqNotFound)50  site:  50  seq len:  203 pep:  ENVASLGTTVRDYTQ
P07203 :   corrected:  (seqNotFound)51  site:  51  seq len:  203 pep:  NVASLGTTVRDYTQM
P07203 :   corrected:  (seqNotFound)55  site:  55  seq len:  203 pep:  LGTTVRDYTQMNELQ
Done. Time	267.639
Formatting  group_12.csv ...
Reading input file...
Get unique substrate sites...
Map unique substrate sites...
Done. Time	282.563
Formatting  group_7.csv ...
Reading input file...
Get unique substrate sites...
Map unique substrate sites...
P04229 :   corrected:  (idNotFound)10  site:  10 pep:  CLKLPGGSCMTALTV
P04229 :   corrected:  (idNotFound)13  site:  13 pep:  LPGGSCMTALTVTLM
P04229 :   corrected:  (idNotFound)16  site:  16 pep:  GSCMTALTVTLMVLS
P04229 :   corrected:  (idNotFound)18  site:  18 p

**Remove unmapped/outdated results**
- Check if there is any unmapped substrate/site due to outdated uniprot sequence records
    - get a list of outdated uniprotID

In [9]:
all_results = glob.glob(GPS_temp_dir_site + '*.csv')
    
updatedSub_li = []
for filename in all_results:
    start = time.time() 
    df_unmapped = pd.read_csv(filename, usecols = ['substrate_id','substrate_acc', 'mapped site'])
    # if the mapped site is not S/T/Y
    df_unmapped = df_unmapped[~(df_unmapped['mapped site'].str.contains('S|T|Y', na=False)) | (df_unmapped['substrate_id'] == 'outdated')]
    df_unmapped = df_unmapped.substrate_acc.drop_duplicates()
    unmapped_li = df_unmapped.values.tolist()
    updatedSub_li.extend(unmapped_li)
    end = time.time()
    print (f"Time\t{(end-start):.3f}")
updatedSub_li 

Time	0.297
Time	9.753
Time	26.570
Time	25.189
Time	26.773
Time	27.444
Time	28.355
Time	27.789
Time	27.169
Time	26.256
Time	27.105
Time	25.683
Time	27.346
Time	26.498
Time	28.012
Time	28.322
Time	28.491
Time	28.384
Time	29.279
Time	25.995
Time	28.823
Time	27.255


['Q29974',
 'P01160',
 'Q95IE3',
 'Q30167',
 'P49895',
 'P13760',
 'P01912',
 'Q16881',
 'P13761',
 'Q5Y7A7',
 'P22352',
 'P36969',
 'Q9C0D9',
 'Q9BQE4',
 'Q86VQ6',
 'P59797',
 'P62341',
 'Q9NZV6',
 'Q9NZV5',
 'Q8IZQ5',
 'Q06124',
 'Q9NNW7',
 'Q8WWX9',
 'Q6QHF9',
 'P18283',
 'Q30134',
 'P20039',
 'Q9TQE0',
 'Q9GIY3',
 'P07203',
 'P04229',
 'P59796',
 'Q92813',
 'P63302',
 'Q9BVL4',
 'P55073',
 'P49908',
 'Q99611']

In [10]:
all_results = glob.glob(GPS_temp_dir_site + '*.csv')

for filename in all_results:
    start = time.time()
    df_mapSite = pd.read_csv(filename)
    # remove the outdated records from df_subMap
    df_update = df_mapSite[~df_mapSite['substrate_acc'].isin(updatedSub_li)]
    df_update.to_csv(filename, chunksize=100000, index=False)
    end = time.time()
    print (f"chunk time\t{(end-start):.3f}")

chunk time	0.340
chunk time	48.055
chunk time	136.465
chunk time	143.126
chunk time	154.167
chunk time	162.273
chunk time	167.112
chunk time	165.188
chunk time	160.292
chunk time	152.254
chunk time	159.712
chunk time	152.817
chunk time	161.710
chunk time	153.559
chunk time	165.560
chunk time	169.556
chunk time	168.166
chunk time	165.006
chunk time	173.175
chunk time	154.519
chunk time	170.991
chunk time	165.149


### Update GPS results
- The next 6 cells is only for NetworKIN results from outdated Human Proteomes sequences (input sequences for GPS prediction an earlier version than the referece human proteome sequence)

1. download the sequence fasta file of the above UniprotID (updatedSub_li) from Uniprot.org, save it as '../Data/Raw/HumanProteome/GPS_updateSub.fasta'. Submit the 'GPS_updateSub.fasta' in GPS again
2. run gps_convert.gps_convert_directory for accession conversion and site mapping for the result file from 'GPS_updateSub.fasta'

In [11]:
convert_type = 'acc'
gps_convert.gps_convert_directory(GPS_update_dir, GPS_kinase, GPS_temp_dir_acc_update, convert_type)

Formatting  GPS5_updateSub ...
Done. Time	3.313


In [12]:
convert_type = 'site'
gps_convert.gps_convert_directory(GPS_temp_dir_acc_update, HP_csv, GPS_temp_dir_site_update, convert_type)

Formatting  GPS5_updateSub.csv ...
Reading input file...
Get unique substrate sites...
Map unique substrate sites...
P49895 :   corrected:  (seqNotFound)123  site:  123  seq len:  249 pep:  PLVLNFGSCTPSFMF
P49895 :   corrected:  (seqNotFound)125  site:  125  seq len:  249 pep:  VLNFGSCTPSFMFKF
P49895 :   corrected:  (seqNotFound)127  site:  127  seq len:  249 pep:  NFGSCTPSFMFKFDQ
Q16881 :   corrected:  (seqNotFound)641  site:  641  seq len:  649 pep:  VTKRSGASILQAGCG
P22352 :   corrected:  (seqNotFound)71  site:  71  seq len:  226 pep:  VLFVNVASYGLTGQY
P22352 :   corrected:  (seqNotFound)75  site:  75  seq len:  226 pep:  NVASYGLTGQYIELN
P36969 :   corrected:  (seqNotFound)67  site:  67  seq len:  197 pep:  RGFVCIVTNVASQGK
P36969 :   corrected:  (seqNotFound)71  site:  71  seq len:  197 pep:  CIVTNVASQGKTEVN
P36969 :   corrected:  (seqNotFound)75  site:  75  seq len:  197 pep:  NVASQGKTEVNYTQL
Q9C0D9 :   corrected:  (seqNotFound)385  site:  385  seq len:  397 pep:  FSLRKPNSDLGMEEK
Q9B

3. save 'GPS_updateSub.csv' under the same dir as other mapped site files
    - GPS ignorgs the 'U'(Selenocysteine) in some substrate sequences, this causes frame shift of the downstream sequences
    - those unmapped sites will not be included

In [13]:
df_mapUpdateSite = pd.read_csv(GPS_temp_dir_site_update + 'GPS_updateSub.csv')
df_mapUpdateSite = df_mapUpdateSite[(df_mapUpdateSite['mapped site'].str.contains('S|T|Y', na=False)) & (df_mapUpdateSite['substrate_id'] != 'outdated')]
df_mapUpdateSite.to_csv(GPS_temp_dir_site + 'GPS_updateSub.csv')

df_mapUpdateSite

,substrate_acc,site,residue,position,predictor,kinase,kinase_acc,pep,score,threshold,mapped site,substrate_id
0,P01160,S2,S,2,AGC/Akt/AKT1,AKT1,P31749,______MSSFSTTTV,-14.300,NaN,S2,P01160_2
1,P01160,S3,S,3,AGC/Akt/AKT1,AKT1,P31749,_____MSSFSTTTVS,-12.700,NaN,S3,P01160_3
2,P01160,S5,S,5,AGC/Akt/AKT1,AKT1,P31749,___MSSFSTTTVSFL,-0.567,NaN,S5,P01160_5
3,P01160,T6,T,6,AGC/Akt/AKT1,AKT1,P31749,__MSSFSTTTVSFLL,4.387,NaN,T6,P01160_6
4,P01160,T7,T,7,AGC/Akt/AKT1,AKT1,P31749,_MSSFSTTTVSFLLL,7.142,NaN,T7,P01160_7
...,...,...,...,...,...,...,...,...,...,...,...,...
288317,Q99611,Y140,Y,140,Dual/Other/WEE/WEE1/WEE1,WEE1,P30291,VQTTDFFYPLVEDPY,-0.525,NaN,Y141,Q99611_141
288318,Q99611,Y147,Y,147,Dual/Other/WEE/WEE1/WEE1,WEE1,P30291,YPLVEDPYMMGRIAC,1.424,NaN,Y148,Q99611_148
288319,Q99611,Y162,Y,162,Dual/Other/WEE/WEE1/WEE1,WEE1,P30291,ANVLSDLYAMGITEC,-0.275,NaN,Y163,Q99611_163
288320,Q99611,Y287,Y,287,Dual/Other/WEE/WEE1/WEE1,WEE1,P30291,REEVELAYQEAMFNM,0.302,NaN,Y288,Q99611_288


### Get the Gene Name of the Substrates from the Reference Human Proteome
- get and add the Gene Name that would use across all perdictors for the substrates to the result files

In [16]:
# get the protein gene names and accessions from the Reference Human Proteome
df_unique_sub = pd.read_csv(HP_csv, usecols = ['UniprotID','Gene Name'], sep = '\t')
df_unique_sub

,Gene Name,UniprotID
0,ACTN1,P12814
1,STAT3,P40763
2,ADD1,P35611
3,ADD2,P35612
4,ADRA2A,P08913
...,...,...
20358,HSFX4,A0A1B0GTS1
20359,TRBJ2-6,A0A0A0MT70
20360,TMEM225B,P0DP42
20361,SMIM29,Q86T20


In [17]:
# add the Gene Name that would use across all perdictors for the substrates to the result files
all_results = glob.glob(GPS_temp_dir_site + '*.csv')

for filename in all_results:
    df = pd.read_csv(filename, usecols = ['substrate_id', 'substrate_acc', 'mapped site', 'predictor', 'kinase', 'kinase_acc', 'pep', 'score'])
    # merge df_subsMap with df_unique_sub to add the common substrate gene name to the df
    df = df.merge(df_unique_sub, left_on=['substrate_acc'], right_on=['UniprotID'], how = 'left')

    df = df.drop(columns = ['UniprotID'])
    df.to_csv(filename, index=False)
df

,substrate_acc,predictor,kinase,kinase_acc,pep,score,mapped site,substrate_id,Gene Name
0,Q9Y6J8,AGC/Akt/AKT1,AKT1,P31749,GLLLCEPTELYNILN,7.126,T10,Q9Y6J8_10,STYXL1
1,Q9Y6J8,AGC/Akt/AKT1,AKT1,P31749,YNILNQATKLSRLTD,2.233,T20,Q9Y6J8_20,STYXL1
2,Q9Y6J8,AGC/Akt/AKT1,AKT1,P31749,LNQATKLSRLTDPNY,4.664,S23,Q9Y6J8_23,STYXL1
3,Q9Y6J8,AGC/Akt/AKT1,AKT1,P31749,ATKLSRLTDPNYLCL,4.142,T26,Q9Y6J8_26,STYXL1
4,Q9Y6J8,AGC/Akt/AKT1,AKT1,P31749,LCLLDVRSKWEYDES,2.135,S38,Q9Y6J8_38,STYXL1
...,...,...,...,...,...,...,...,...,...
20762757,Q9HA92,Dual/Other/WEE/WEE1/WEE1,WEE1,P30291,CDDHLSLYQLSLERG,0.400,Y226,Q9HA92_226,RSAD1
20762758,Q9HA92,Dual/Other/WEE/WEE1/WEE1,WEE1,P30291,PELAAEMYQRGRAVL,-0.158,Y257,Q9HA92_257,RSAD1
20762759,Q9HA92,Dual/Other/WEE/WEE1/WEE1,WEE1,P30291,REAGFHQYEVSNFAR,0.464,Y272,Q9HA92_272,RSAD1
20762760,Q9HA92,Dual/Other/WEE/WEE1/WEE1,WEE1,P30291,LSTHNWTYWQCGQYL,0.479,Y290,Q9HA92_290,RSAD1


### Creating Resource Files
**globalKinaseMap**
- creat a new or add unique kinases from NetworKIN to the globel kinase resource file.
- get and add the Kinase Name that would use across all perdictors for the kinases to the result files

In [18]:
# get unique kinases in the NetworKIN result files

all_results = glob.glob(GPS_temp_dir_site + '*.csv')

df_unique_kin = pd.DataFrame()

for filename in all_results:
    df = pd.read_csv(filename, usecols = ['kinase_acc', 'kinase'])
    df = df.drop_duplicates()
    df_unique_kin = df_unique_kin.append(df, ignore_index=True)
    
df_unique_kin = df_unique_kin.drop_duplicates()
df_unique_kin

,kinase,kinase_acc
0,AKT1,P31749
1,AKT2,P31751
2,AKT3,Q9Y243
3,PDPK1,O15530
4,PKACA,P17612
...,...,...
321,LYN,P07948
322,BAZ1B,Q9UIG0
323,CSNK2A1,P68400
324,WEE1B,P0C1S8


In [5]:
unmapped_list = pd.DataFrame()

if os.path.isfile(KinaseMap): 
    df_humanKinase = pd.read_csv(KinaseMap)
# if globalkinaseMap.csv file does not exist, create an new df using orginal human kinase map
else:
    df_humanKinase = pd.read_csv(HK_org, usecols = ['Kinase Name', 'Preferred Name', 'UniprotID', 'Type', 'description'], sep = '\t')
    df_humanKinase['description'].replace(regex=True,inplace=True,to_replace=r'\[Source.+\]',value=r'')

for index, row in df_unique_kin.iterrows():
    kinase = df_unique_kin.at[index, 'kinase_acc']
    # if the kinase/other enzyme already in the globalKinaseMap.csv file
    if any(df_humanKinase['UniprotID'] == kinase):
        # get the index of the substrate in the globalKinaseMap.csv file 
        idx = df_humanKinase.index[df_humanKinase.UniprotID == kinase].values[0] 

        df_humanKinase.at[idx, 'GPS5_kinase_name'] = df_unique_kin.at[index, 'kinase']
        
    # if the kinase is not in the globalSubstrateMap.csv file, we need a list to check annotations manullay
    else:
        unmapped_list = unmapped_list.append(row,sort=False).reset_index(drop=True)
        
print (unmapped_list)
    
df_humanKinase 


,Kinase Name,Preferred Name,UniprotID,Type,description,GPS5_kinase_name,NetworKIN_kinase_name,PhosphoPICK_kinase_name
0,AAK1,AAK1,Q2M2I8,Pkinase,AP2 associated kinase 1,AAK1,NaN,NaN
1,ABL1,ABL1,P00519,Pkinase_tyr,"ABL proto-oncogene 1, non-receptor tyrosine ki...",ABL1,ABL1,ABL1
2,ABL2,ABL2,P42684,Pkinase_tyr,"ABL proto-oncogene 2, non-receptor tyrosine ki...",ABL2,ABL2,ABL2
3,ACVR1,ACVR1,Q04771,Pkinase,activin A receptor type 1,NaN,NaN,NaN
4,ACVR1B,ACVR1B,P36896,Pkinase,activin A receptor type 1B,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
484,WNK2,WNK2,Q9Y3S1,Pkinase,WNK lysine deficient protein kinase 2,WNK2,NaN,NaN
485,WNK3,WNK3,Q9BYP7,Pkinase,WNK lysine deficient protein kinase 3,WNK3,NaN,NaN
486,WNK4,WNK4,Q96J92,Pkinase,WNK lysine deficient protein kinase 4,WNK4,NaN,NaN
487,YES1,YES1,P07947,Pkinase_tyr,"YES proto-oncogene 1, Src family tyrosine kinase",YES1,YES1,NaN


- After manully searched in uniprot records for the aboved 7 unmapped kinases/enzymes, 1 is annotated as protein kinase. These are added to the final **globalKinaseMap.csv**. The other 5 that are not protein kinase are removed from the **df_kinaseMap**.

In [22]:
not_kinase = ['PDK2', 'PDK3', 'PDK4', 'MSN', 'GTF2F1', 'MPS1']

new_kinase = {'BCKDK':'[3-methyl-2-oxobutanoate dehydrogenase [lipoamide]] kinase, mitochondrial'}

- add the 1 protein kinases to the globalKinaseMap.csv using above dictionary 

In [6]:
# get the length (last index) of the current df_humanKinase
len = df_humanKinase.UniprotID.count()

# add the 16 new kinase in the globalKinaseMap.csv
for key in new_kinase:
    # add the kinase name and description for the new kinase 
    df_humanKinase = df_humanKinase.append({'Kinase Name': key}, ignore_index=True)
    df_humanKinase.at[len,'Preferred Name'] = key
    df_humanKinase.at[len,'description'] = new_kinase[key]
    # get the index of where the new kinase is in the df_unique_kinase
    i = df_unique_kinase.index[df_unique_kinase['kinase'] == key].values[0] 
    # add the uniprotID for the new kinase 
    df_humanKinase.at[len,'UniprotID'] = df_unique_kinase.at[i, 'kinase_acc']
    # add the accs used in HPRD for the new kinase
    df_humanKinase.at[len,'GPS5_kinase_name'] = df_unique_kinase.at[i, 'kinase']
    
    len += 1
    
df_humanKinase.to_csv(KinaseMap,index=False) 
df_humanKinase

,Kinase Name,Preferred Name,UniprotID,Type,description,GPS5_kinase_name,NetworKIN_kinase_name,PhosphoPICK_kinase_name
0,AAK1,AAK1,Q2M2I8,Pkinase,AP2 associated kinase 1,AAK1,NaN,NaN
1,ABL1,ABL1,P00519,Pkinase_tyr,"ABL proto-oncogene 1, non-receptor tyrosine ki...",ABL1,ABL1,ABL1
2,ABL2,ABL2,P42684,Pkinase_tyr,"ABL proto-oncogene 2, non-receptor tyrosine ki...",ABL2,ABL2,ABL2
3,ACVR1,ACVR1,Q04771,Pkinase,activin A receptor type 1,NaN,NaN,NaN
4,ACVR1B,ACVR1B,P36896,Pkinase,activin A receptor type 1B,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
484,WNK2,WNK2,Q9Y3S1,Pkinase,WNK lysine deficient protein kinase 2,WNK2,NaN,NaN
485,WNK3,WNK3,Q9BYP7,Pkinase,WNK lysine deficient protein kinase 3,WNK3,NaN,NaN
486,WNK4,WNK4,Q96J92,Pkinase,WNK lysine deficient protein kinase 4,WNK4,NaN,NaN
487,YES1,YES1,P07947,Pkinase_tyr,"YES proto-oncogene 1, Src family tyrosine kinase",YES1,YES1,NaN


In [23]:
# get the new list kinase with common kinase name that would use across all referece and the uniprotID for these kinase
df_unique_kin = df_humanKinase[['Kinase Name','UniprotID']]
df_unique_kin

,Kinase Name,UniprotID
0,SGK1,O00141
1,BMPR1B,O00238
2,CDC7,O00311
3,PLK4,O00444
4,STK25,O00506
...,...,...
484,TP53RK,Q96S44
485,TRPM6,Q9BX84
486,BCR/ABL,A9UF07
487,BCKDK,O14874


- remove the 6 enzyme records that are not protein kinase from the df_kinaseMap
- add the Kinase Name that would use across all perdictors for the kinase to the result files

In [24]:
all_results = glob.glob(GPS_temp_dir_site + '*.csv')

for filename in all_results:
    df = pd.read_csv(filename)
    # remove the 4 enzyme records that are not protein kinase from the df_kinaseMap
    df = df[~df['kinase'].isin(not_kinase)]

    # merge with df_unique_kinase to add the common kinase name to the df
    df = df.merge(df_unique_kin, left_on='kinase_acc', right_on='UniprotID', how = 'left')
    # drop the duplicated uniprot id for kinases
    df = df.drop(columns = 'UniprotID')

    df.to_csv(filename,index=False)  

df

,substrate_acc,predictor,kinase,kinase_acc,pep,score,mapped site,substrate_id,Gene Name,Kinase Name
0,Q9Y6J8,AGC/Akt/AKT1,AKT1,P31749,GLLLCEPTELYNILN,7.126,T10,Q9Y6J8_10,STYXL1,AKT1
1,Q9Y6J8,AGC/Akt/AKT1,AKT1,P31749,YNILNQATKLSRLTD,2.233,T20,Q9Y6J8_20,STYXL1,AKT1
2,Q9Y6J8,AGC/Akt/AKT1,AKT1,P31749,LNQATKLSRLTDPNY,4.664,S23,Q9Y6J8_23,STYXL1,AKT1
3,Q9Y6J8,AGC/Akt/AKT1,AKT1,P31749,ATKLSRLTDPNYLCL,4.142,T26,Q9Y6J8_26,STYXL1,AKT1
4,Q9Y6J8,AGC/Akt/AKT1,AKT1,P31749,LCLLDVRSKWEYDES,2.135,S38,Q9Y6J8_38,STYXL1,AKT1
...,...,...,...,...,...,...,...,...,...,...
20303283,Q9HA92,Dual/Other/WEE/WEE1/WEE1,WEE1,P30291,CDDHLSLYQLSLERG,0.400,Y226,Q9HA92_226,RSAD1,WEE1
20303284,Q9HA92,Dual/Other/WEE/WEE1/WEE1,WEE1,P30291,PELAAEMYQRGRAVL,-0.158,Y257,Q9HA92_257,RSAD1,WEE1
20303285,Q9HA92,Dual/Other/WEE/WEE1/WEE1,WEE1,P30291,REAGFHQYEVSNFAR,0.464,Y272,Q9HA92_272,RSAD1,WEE1
20303286,Q9HA92,Dual/Other/WEE/WEE1/WEE1,WEE1,P30291,LSTHNWTYWQCGQYL,0.479,Y290,Q9HA92_290,RSAD1,WEE1


# Standard Formatted GPS
### 'GPS_formatted.csv'
Standardize the preprocessed file with following columns:
- **substrate_id** - unique IDs for the substrate phosphorylation site (substrate_acc + position)
- **substrate_name** - gene name for the substrates
- **substrate_acc** - mapped UniprotIDs for the substrates
- **site** - phosphorylation  site
- **pep** - +/- 7 AA peptide sequence around the site
- **score** - perdiction score
- **Kinase Name** - Kinase name

In [4]:
all_results = glob.glob(GPS_temp_dir_site + '*.csv')
GPS = []
for filename in all_results:
    df = pd.read_csv(filename, usecols = ['substrate_id','substrate_acc','Gene Name', 'mapped site','pep', 'score', 'Kinase Name'])
    df = df.rename(columns={'Gene Name': 'substrate_name', 'mapped site' : 'site'})

    if not os.path.isfile(GPS_formatted):
        df.to_csv(GPS_formatted, mode='a', index=False, sep=',')
    else:
        df.to_csv(GPS_formatted, mode='a', index=False, sep=',', header=False)


In [ ]:
df_final = pd.read_csv(GPS_formatted)
df_final = df_final.drop_duplicates()
df_final.to_csv(GPS_formatted, chunksize = 1000000, index = False)